In [3]:
import cv2
import os
import multiprocessing
from concurrent.futures import ThreadPoolExecutor

In [5]:
# Input and output directories
video_folder = "/Users/srujansadineni/Desktop/Video/video"
frames_output_folder = "/Users/srujansadineni/Desktop/Video/white"
grayscale_output_folder = "/Users/srujansadineni/Desktop/Video/black"


In [7]:
# Create output directories
os.makedirs(frames_output_folder, exist_ok=True)
os.makedirs(grayscale_output_folder, exist_ok=True)

use_cuda = cv2.cuda.getCudaEnabledDeviceCount() > 0

def process_frame(frame):
    if use_cuda:
        gpu_frame = cv2.cuda_GpuMat()
        gpu_frame.upload(frame)  
        gray_gpu = cv2.cuda.cvtColor(gpu_frame, cv2.COLOR_BGR2GRAY)  
        gray_frame = gray_gpu.download() 
    else:
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  

    return gray_frame

def process_video(video_path):
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    frame_folder = os.path.join(frames_output_folder, video_name)
    gray_folder = os.path.join(grayscale_output_folder, video_name)

    os.makedirs(frame_folder, exist_ok=True)
    os.makedirs(gray_folder, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    
    frame_count = 0
    saved_count = 0
    frame_skip = 3  # Process every 3rd frame for speed

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_skip == 0:  
            frame_filename = os.path.join(frame_folder, f"frame_{saved_count:04d}.jpg")
            cv2.imwrite(frame_filename, frame, [cv2.IMWRITE_JPEG_QUALITY, 90])  

            # Convert frame to grayscale (fast)
            gray_image = process_frame(frame)
            gray_filename = os.path.join(gray_folder, f"gray_{saved_count:04d}.jpg")
            cv2.imwrite(gray_filename, gray_image, [cv2.IMWRITE_JPEG_QUALITY, 90])

            saved_count += 1

        frame_count += 1

    cap.release()
    print(f"✅ Processed: {video_name} - Saved {saved_count} frames")

# all video files in folder
video_files = [os.path.join(video_folder, f) for f in os.listdir(video_folder) if f.endswith(('.mp4', '.avi', '.mov'))]

# Use multiprocessing to process videos in parallel
if __name__ == "__main__":
    with ThreadPoolExecutor(max_workers=6) as executor:  
        executor.map(process_video, video_files)

print("✅ All videos processed successfully!")


✅ Processed: Geometric shapes and their light and shadow forms - Saved 4018 frames
✅ All videos processed successfully!
